In [2]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from deepxde.backend import tf

# Define Navier-Stokes equations for 2D steady-state incompressible flow
def navier_stokes(pde_input, u, v, p, Re=100):
    x, y = pde_input[:, 0:1], pde_input[:, 1:2]

    u_x = dde.grad.jacobian(u, pde_input, i=0, j=0)
    u_y = dde.grad.jacobian(u, pde_input, i=0, j=1)
    v_x = dde.grad.jacobian(v, pde_input, i=0, j=0)
    v_y = dde.grad.jacobian(v, pde_input, i=0, j=1)
    p_x = dde.grad.jacobian(p, pde_input, i=0, j=0)
    p_y = dde.grad.jacobian(p, pde_input, i=0, j=1)

    u_xx = dde.grad.hessian(u, pde_input, i=0, j=0)
    u_yy = dde.grad.hessian(u, pde_input, i=1, j=1)
    v_xx = dde.grad.hessian(v, pde_input, i=0, j=0)
    v_yy = dde.grad.hessian(v, pde_input, i=1, j=1)

    continuity_eq = u_x + v_y
    momentum_x = u * u_x + v * u_y + p_x - (1 / Re) * (u_xx + u_yy)
    momentum_y = u * v_x + v * v_y + p_y - (1 / Re) * (v_xx + v_yy)

    return [continuity_eq, momentum_x, momentum_y]

# Define the geometry of the circular airfoil
def circular_airfoil():
    def boundary(x, _):
        return np.sqrt(x[:, 0] ** 2 + x[:, 1] ** 2) < 1.0

    geom = dde.geometry.Rectangle([-2, -1], [4, 1])
    circle = dde.geometry.Disk([0, 0], 1)
    domain = geom - circle
    return domain, boundary

# Define PINN model
def train_pinn():
    domain, boundary = circular_airfoil()
    data = dde.data.PDE(domain, navier_stokes, [], num_domain=10000, num_boundary=2000)

    net = dde.maps.FNN([2] + [50] * 4 + [3], "tanh", "Glorot normal")
    model = dde.Model(data, net)
    model.compile("adam", lr=0.001)

    loss_history, train_state = model.train(iterations=10000)
    dde.saveplot(loss_history, train_state, issave=True, isplot=True)
    return model

# Train and evaluate PINN model
model = train_pinn()

# Predict velocity and pressure fields
n_test = 1000
test_points = np.random.uniform([-2, -1], [4, 1], size=(n_test, 2))
flow_field = model.predict(test_points)

# Plot velocity field
plt.figure(figsize=(8, 6))
plt.quiver(test_points[:, 0], test_points[:, 1], flow_field[:, 0], flow_field[:, 1], color='blue')
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.title("Predicted Velocity Field Over Airfoil")
plt.grid(True)
plt.show()

# Compute performance metrics
r2 = dde.metrics.r2_score(np.zeros_like(flow_field[:, 2]), flow_field[:, 2])
mse = dde.metrics.mean_squared_error(np.zeros_like(flow_field[:, 2]), flow_field[:, 2])
print(f"Model Performance: R² = {r2:.4f}, MSE = {mse:.6f}")


ModuleNotFoundError: No module named 'deepxde'